## The functionalities in this notebook is as follows:

1. Given a graph, calculate the kernel matrix - diffusion for example

2. Sample from a GP with this kernel matrix, for a given hyper parameters

3. fit another GP with the same kernel  (and unknown hyperparamters)

### Example: Sample data from a GP on graph

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
from ipywidgets import FloatSlider, interactive

def diffusion_kernel(adj_matrix, beta):
    laplacian = np.diag(np.sum(adj_matrix, axis=1)) - adj_matrix  # Graph Laplacian
    return expm(-beta * laplacian)  # Matrix exponential

def plot_sampled_values(beta):
    num_nodes = 20
    adjacency_matrix = np.eye(num_nodes, k=1) + np.eye(num_nodes, k=-1)  # Circular adjacency matrix
    K = diffusion_kernel(adjacency_matrix, beta)
    L = np.linalg.cholesky(K + 1e-6 * np.eye(num_nodes))  # Cholesky decomposition
    samples = L @ np.random.normal(size=(num_nodes, 1))  # Sample from Gaussian process

    # Create a figure with two subplots
    fig, axs = plt.subplots(1, 2, figsize=(15, 6))

    # Plot sampled values
    axs[0].plot(range(num_nodes), samples.flatten(), marker='o', linestyle='-', color='b')
    axs[0].set_title(f'Sampled Values for Each Node in Circular Graph (beta={beta:.2f})')
    axs[0].set_xlabel('Node Number')
    axs[0].set_ylabel('Sampled Value')
    axs[0].set_xticks(range(num_nodes))
    axs[0].grid()

    # Plot kernel matrix as a heatmap
    cax = axs[1].matshow(K, cmap='viridis')  # Heatmap of the kernel matrix
    axs[1].set_title(f'Kernel Matrix Heatmap (beta={beta:.2f})')
    plt.colorbar(cax, ax=axs[1])
    axs[1].set_xlabel('Node Index')
    axs[1].set_ylabel('Node Index')

    plt.show()

# Interactive slider for beta
beta_slider = FloatSlider(value=2.0, min=0.1, max=10.0, step=0.1, description='Beta (σ):')
interactive_plot = interactive(plot_sampled_values, beta=beta_slider)
interactive_plot


interactive(children=(FloatSlider(value=2.0, description='Beta (σ):', max=10.0, min=0.1), Output()), _dom_clas…

### Fit the sampled data in a new GP

1. define a kernel compatible with GPflow, with tunable parameters

2. learn the hyper-parameters by maximizing log likelihood

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
import gpflow
import tensorflow as tf
import ipywidgets as widgets
from IPython.display import display

# Function to compute the diffusion kernel
def diffusion_kernel(adj_matrix, beta):
    laplacian = np.diag(np.sum(adj_matrix, axis=1)) - adj_matrix  # Graph Laplacian
    return expm(-beta * laplacian)  # Matrix exponential

# Parameters
num_nodes = 50
adjacency_matrix = np.eye(num_nodes, k=1) + np.eye(num_nodes, k=-1)  # Circular adjacency matrix

# Generate noisy samples function
def generate_noisy_samples(beta_sample):
    K = diffusion_kernel(adjacency_matrix, beta_sample)
    L = np.linalg.cholesky(K + 1e-6 * np.eye(num_nodes))  # Cholesky decomposition
    true_samples = L @ np.random.normal(size=(num_nodes, 1))  # Sample from Gaussian process
    noise = 0.1 * np.random.randn(num_nodes, 1)  # Additive noise
    Y_noisy = true_samples + noise  # Noisy observations
    return Y_noisy

# Modified `K` function in GraphDiffusionKernel class
class GraphDiffusionKernel(gpflow.kernels.Kernel):
    def __init__(self, adjacency_matrix, **kwargs):
        super().__init__(**kwargs)
        self.adjacency_matrix = tf.convert_to_tensor(adjacency_matrix, dtype=tf.float64)
        self.beta = gpflow.Parameter(2.0, transform=gpflow.utilities.positive())  # Learnable hyperparameter

    def K(self, X1, X2=None):
        if X2 is None:
            X2 = X1
        # Compute the full diffusion kernel
        kernel_matrix = self.diffusion_kernel(self.adjacency_matrix, self.beta)
        # Select the appropriate submatrix based on X1 and X2 indices
        indices_X1 = tf.cast(tf.reshape(X1, [-1]), dtype=tf.int32)
        indices_X2 = tf.cast(tf.reshape(X2, [-1]), dtype=tf.int32)
        return tf.gather(tf.gather(kernel_matrix, indices_X1, axis=0), indices_X2, axis=1)

    def K_diag(self, X):
        kernel_matrix = self.diffusion_kernel(self.adjacency_matrix, self.beta)
        indices_X = tf.cast(tf.reshape(X, [-1]), dtype=tf.int32)
        return tf.gather(tf.linalg.diag_part(kernel_matrix), indices_X)

    def diffusion_kernel(self, adj_matrix, beta):
        laplacian = tf.linalg.diag(tf.reduce_sum(adj_matrix, axis=1)) - adj_matrix  # Graph Laplacian
        return tf.linalg.expm(-beta * laplacian)

# Function to plot the results
def plot_results(beta_sample):
    # Generate noisy samples
    Y_noisy = generate_noisy_samples(beta_sample)

    # Convert noisy data to TensorFlow tensor
    X = tf.convert_to_tensor(np.arange(num_nodes, dtype=np.float64).reshape(-1, 1))  # Input features (nodes)
    Y = tf.convert_to_tensor(Y_noisy, dtype=tf.float64)  # Noisy sampled data

    # Create an instance of the kernel
    graph_kernel = GraphDiffusionKernel(adjacency_matrix)

    # GPflow model
    model = gpflow.models.GPR(data=(X, Y), kernel=graph_kernel, mean_function=None)

    # Optimize the model
    gpflow.optimizers.Scipy().minimize(model.training_loss, model.trainable_variables)

    # Prediction for visualization
    X_new = tf.convert_to_tensor(np.arange(num_nodes).reshape(-1, 1), dtype=tf.float64)  # New input features for prediction
    mean, variance = model.predict_f(X_new)  # Predict mean and variance
    stddev = tf.sqrt(variance)  # Standard deviation

    # Plotting
    plt.figure(figsize=(12, 8))
    plt.plot(X.numpy(), Y.numpy(), 'ro', label='Noisy Samples')  # Sampled data points
    plt.plot(X_new.numpy(), mean.numpy(), 'b-', label='Fitted Mean')  # Fitted mean
    plt.fill_between(X_new.numpy().flatten(), 
                     (mean - 1.96 * stddev).numpy().flatten(), 
                     (mean + 1.96 * stddev).numpy().flatten(), 
                     color='lightblue', alpha=0.5, label='95% Confidence Interval')  # 95% confidence interval
    plt.title(f'Gaussian Process Fit with Graph Diffusion Kernel (beta={beta_sample})')
    plt.xlabel('Node Number')
    plt.ylabel('Sampled Value')
    plt.xticks(range(num_nodes))
    plt.grid()
    plt.legend()
    plt.show()
    print("Learned beta:", model.kernel.beta.numpy())

# Create a slider for beta
beta_slider = widgets.FloatSlider(value=3.0, min=0.1, max=10.0, step=0.1, description='Beta:')
ui = widgets.VBox([beta_slider])

# Link the slider to the plotting function
out = widgets.interactive_output(plot_results, {'beta_sample': beta_slider})
display(ui, out)


Output()

### Visualizing the (linear) graph we fit and the fitted GP

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
import gpflow
import tensorflow as tf
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objs as go
import networkx as nx

# Function to compute the diffusion kernel
def diffusion_kernel(adj_matrix, beta):
    laplacian = np.diag(np.sum(adj_matrix, axis=1)) - adj_matrix  # Graph Laplacian
    return expm(-beta * laplacian)  # Matrix exponential

# Parameters
num_nodes = 40
adjacency_matrix = np.eye(num_nodes, k=1) + np.eye(num_nodes, k=-1)  # Circular adjacency matrix

# Generate noisy samples function
def generate_noisy_samples(beta_sample):
    K = diffusion_kernel(adjacency_matrix, beta_sample)
    L = np.linalg.cholesky(K + 1e-6 * np.eye(num_nodes))  # Cholesky decomposition
    true_samples = L @ np.random.normal(size=(num_nodes, 1))  # Sample from Gaussian process
    noise = 0.1 * np.random.randn(num_nodes, 1)  # Additive noise
    Y_noisy = true_samples + noise  # Noisy observations
    return Y_noisy

# Modified `K` function in GraphDiffusionKernel class
class GraphDiffusionKernel(gpflow.kernels.Kernel):
    def __init__(self, adjacency_matrix, **kwargs):
        super().__init__(**kwargs)
        self.adjacency_matrix = tf.convert_to_tensor(adjacency_matrix, dtype=tf.float64)
        self.beta = gpflow.Parameter(2.0, transform=gpflow.utilities.positive())  # Learnable hyperparameter

    def K(self, X1, X2=None):
        if X2 is None:
            X2 = X1
        kernel_matrix = self.diffusion_kernel(self.adjacency_matrix, self.beta)
        indices_X1 = tf.cast(tf.reshape(X1, [-1]), dtype=tf.int32)
        indices_X2 = tf.cast(tf.reshape(X2, [-1]), dtype=tf.int32)
        return tf.gather(tf.gather(kernel_matrix, indices_X1, axis=0), indices_X2, axis=1)

    def K_diag(self, X):
        kernel_matrix = self.diffusion_kernel(self.adjacency_matrix, self.beta)
        indices_X = tf.cast(tf.reshape(X, [-1]), dtype=tf.int32)
        return tf.gather(tf.linalg.diag_part(kernel_matrix), indices_X)

    def diffusion_kernel(self, adj_matrix, beta):
        laplacian = tf.linalg.diag(tf.reduce_sum(adj_matrix, axis=1)) - adj_matrix  # Graph Laplacian
        return tf.linalg.expm(-beta * laplacian)

# Function to plot the results
def plot_results(beta_sample):
    clear_output(wait=True)  # Clear previous output
    # Generate noisy samples
    Y_noisy = generate_noisy_samples(beta_sample)

    # Convert noisy data to TensorFlow tensor
    X = tf.convert_to_tensor(np.arange(num_nodes, dtype=np.float64).reshape(-1, 1))  # Input features (nodes)
    Y = tf.convert_to_tensor(Y_noisy, dtype=tf.float64)  # Noisy sampled data

    # Create an instance of the kernel
    graph_kernel = GraphDiffusionKernel(adjacency_matrix)

    # GPflow model
    model = gpflow.models.GPR(data=(X, Y), kernel=graph_kernel, mean_function=None)

    # Optimize the model
    gpflow.optimizers.Scipy().minimize(model.training_loss, model.trainable_variables)

    # Prediction for visualization
    X_new = tf.convert_to_tensor(np.arange(num_nodes).reshape(-1, 1), dtype=tf.float64)  # New input features for prediction
    mean, variance = model.predict_f(X_new)  # Predict mean and variance
    stddev = tf.sqrt(variance)  # Standard deviation

    # Plotting the Gaussian Process results
    fig, ax = plt.subplots(1, 2, figsize=(16, 6))

    # GP plot on the left
    ax[0].plot(X.numpy(), Y.numpy(), 'ro', label='Noisy Samples')
    ax[0].plot(X_new.numpy(), mean.numpy(), 'b-', label='Fitted Mean')
    ax[0].fill_between(X_new.numpy().flatten(),
                       (mean - 1.96 * stddev).numpy().flatten(),
                       (mean + 1.96 * stddev).numpy().flatten(),
                       color='lightblue', alpha=0.5, label='95% Confidence Interval')
    ax[0].set_title(f'Gaussian Process Fit with Graph Diffusion Kernel (beta={beta_sample})')
    ax[0].set_xlabel('Node Number')
    ax[0].set_ylabel('Sampled Value')
    ax[0].grid()
    ax[0].legend()
    
    # Plot the network graph using NetworkX and Plotly
    plot_network_graph(adjacency_matrix, ax[1])

    plt.show()
    print("Learned beta:", model.kernel.beta.numpy())

# Function to plot the network graph in Matplotlib
def plot_network_graph(adjacency_matrix, ax):
    # Create a NetworkX graph from the adjacency matrix
    G = nx.from_numpy_array(adjacency_matrix)
    pos = nx.spring_layout(G)
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos, ax=ax, node_size=300, node_color='red')
    
    # Draw edges
    nx.draw_networkx_edges(G, pos, ax=ax, edge_color='blue')
    
    ax.set_title("Network Graph Representation of Adjacency Matrix")
    ax.axis("off")

# Create a slider for beta
beta_slider = widgets.FloatSlider(value=3.0, min=0.1, max=10.0, step=0.1, description='Beta:')
ui = widgets.VBox([beta_slider])

# Link the slider to the plotting function
out = widgets.interactive_output(plot_results, {'beta_sample': beta_slider})
display(ui, out)


Output()

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
import gpflow
import tensorflow as tf
import ipywidgets as widgets
from IPython.display import display, clear_output
import networkx as nx

# Function to compute the diffusion kernel
def diffusion_kernel(adj_matrix, beta):
    laplacian = np.diag(np.sum(adj_matrix, axis=1)) - adj_matrix  # Graph Laplacian
    return expm(-beta * laplacian)  # Matrix exponential

# Parameters
num_nodes = 100
# Generate an undirected random graph
probability = 0.03  # Probability of edge creation
G = nx.erdos_renyi_graph(num_nodes, probability, directed=False)  # Ensure the graph is undirected
adjacency_matrix = nx.to_numpy_array(G)  # Convert to adjacency matrix

# Generate noisy samples function
def generate_noisy_samples(beta_sample):
    K = diffusion_kernel(adjacency_matrix, beta_sample)
    L = np.linalg.cholesky(K + 1e-6 * np.eye(num_nodes))  # Cholesky decomposition
    true_samples = L @ np.random.normal(size=(num_nodes, 1))  # Sample from Gaussian process
    noise = 0.1 * np.random.randn(num_nodes, 1)  # Additive noise
    Y_noisy = true_samples + noise  # Noisy observations
    return Y_noisy

# Modified `K` function in GraphDiffusionKernel class
class GraphDiffusionKernel(gpflow.kernels.Kernel):
    def __init__(self, adjacency_matrix, **kwargs):
        super().__init__(**kwargs)
        self.adjacency_matrix = tf.convert_to_tensor(adjacency_matrix, dtype=tf.float64)
        self.beta = gpflow.Parameter(2.0, transform=gpflow.utilities.positive())  # Learnable hyperparameter

    def K(self, X1, X2=None):
        if X2 is None:
            X2 = X1
        # Compute the full diffusion kernel
        kernel_matrix = self.diffusion_kernel(self.adjacency_matrix, self.beta)
        indices_X1 = tf.cast(tf.reshape(X1, [-1]), dtype=tf.int32)
        indices_X2 = tf.cast(tf.reshape(X2, [-1]), dtype=tf.int32)
        return tf.gather(tf.gather(kernel_matrix, indices_X1, axis=0), indices_X2, axis=1)

    def K_diag(self, X):
        kernel_matrix = self.diffusion_kernel(self.adjacency_matrix, self.beta)
        indices_X = tf.cast(tf.reshape(X, [-1]), dtype=tf.int32)
        return tf.gather(tf.linalg.diag_part(kernel_matrix), indices_X)

    def diffusion_kernel(self, adj_matrix, beta):
        laplacian = tf.linalg.diag(tf.reduce_sum(adj_matrix, axis=1)) - adj_matrix  # Graph Laplacian
        return tf.linalg.expm(-beta * laplacian)

# Function to plot the results
def plot_results(beta_sample):
    clear_output(wait=True)  # Clear previous output
    # Generate noisy samples
    Y_noisy = generate_noisy_samples(beta_sample)

    # Convert noisy data to TensorFlow tensor
    X = tf.convert_to_tensor(np.arange(num_nodes, dtype=np.float64).reshape(-1, 1))  # Input features (nodes)
    Y = tf.convert_to_tensor(Y_noisy, dtype=tf.float64)  # Noisy sampled data

    # Create an instance of the kernel
    graph_kernel = GraphDiffusionKernel(adjacency_matrix)

    # GPflow model
    model = gpflow.models.GPR(data=(X, Y), kernel=graph_kernel, mean_function=None)

    # Optimize the model
    gpflow.optimizers.Scipy().minimize(model.training_loss, model.trainable_variables)

    # Prediction for visualization
    X_new = tf.convert_to_tensor(np.arange(num_nodes).reshape(-1, 1), dtype=tf.float64)  # New input features for prediction
    mean, variance = model.predict_f(X_new)  # Predict mean and variance
    stddev = tf.sqrt(variance)  # Standard deviation

    # Create side-by-side plots for GP and network graph
    fig, ax = plt.subplots(1, 2, figsize=(16, 6))

    # Gaussian Process plot on the left
    ax[0].plot(X.numpy(), Y.numpy(), 'ro', label='Noisy Samples')
    ax[0].plot(X_new.numpy(), mean.numpy(), 'b-', label='Fitted Mean')
    ax[0].fill_between(X_new.numpy().flatten(),
                       (mean - 1.96 * stddev).numpy().flatten(),
                       (mean + 1.96 * stddev).numpy().flatten(),
                       color='lightblue', alpha=0.5, label='95% Confidence Interval')
    ax[0].set_title(f'Gaussian Process Fit with Graph Diffusion Kernel (beta={beta_sample})')
    ax[0].set_xlabel('Node Number')
    ax[0].set_ylabel('Sampled Value')
    ax[0].grid()
    ax[0].legend()
    
    # Network graph plot on the right
    plot_network_graph(adjacency_matrix, ax[1])

    plt.show()
    print("Learned beta:", model.kernel.beta.numpy())

# Function to plot the network graph in Matplotlib
def plot_network_graph(adjacency_matrix, ax):
    # Create a NetworkX graph from the adjacency matrix
    G = nx.from_numpy_array(adjacency_matrix)
    pos = nx.spring_layout(G)
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos, ax=ax, node_size=300, node_color='red')
    
    # Draw edges
    nx.draw_networkx_edges(G, pos, ax=ax, edge_color='blue')
    
    ax.set_title("Network Graph Representation of Adjacency Matrix")
    ax.axis("off")

# Create a slider for beta
beta_slider = widgets.FloatSlider(value=3.0, min=0.1, max=10.0, step=0.1, description='Beta:')
ui = widgets.VBox([beta_slider])

# Link the slider to the plotting function
out = widgets.interactive_output(plot_results, {'beta_sample': beta_slider})
display(ui, out)


Output()

### Other Notes

In [8]:
# G py

# GPflow

# GP Jax


# constained optimization - positive values - minimize() still works



# f (modulation function) exist  - postitive definite kernel

SyntaxError: invalid syntax (754214315.py, line 1)